In [5]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2

In [6]:
import gspread
from google.colab import auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials
import numpy as np
from scipy.stats import dirichlet
import statistics

# Authenticate the user
auth.authenticate_user()

import google.auth

# Get the authenticated credentials
creds, _ = google.auth.default()

# Authorize the gspread client with the credentials
client = gspread.authorize(creds)


In [7]:
# Replace with your Google Sheets URL
sheet_url = "https://docs.google.com/spreadsheets/d/1J1naRqGMA9R7LmnuybaedTQTWltqvXeFpgsnHSFWyoU/edit?gid=0#gid=0"

# Open the Google Sheet
sheet = client.open_by_url(sheet_url).sheet1  # Access the first worksheet

In [28]:
def read_parametersA():
    our_choiceA = int(sheet.acell('A6').value)
    our_choiceB = int(sheet.acell('A7').value)

    total_rolls = 13
    rounds_revealed = int(sheet.acell('A10').value)
    other_choices_probs_str = sheet.acell('A3').value
    other_choices_probs = [float(prob) for prob in other_choices_probs_str.split(',')]
    teams = int(sheet.acell('C3').value)

    observed_rolls_A = []
    observed_rolls_B = []
    for i in range(rounds_revealed):
        observed_rolls_A.append(int(sheet.acell(f'D{7+i}').value))

    for i in range(rounds_revealed):
        observed_rolls_B.append(int(sheet.acell(f'D{23+i}').value))

    # Validate probabilities
    if len(other_choices_probs) != 6:
        raise ValueError("Exactly 6 probabilities required.")
    if not np.isclose(sum(other_choices_probs), 1.0):
        raise ValueError("Probabilities must sum to 1.")
    return our_choiceA, our_choiceB, total_rolls, rounds_revealed, other_choices_probs, observed_rolls_A, observed_rolls_B

def bayesian_update(observed_rolls, prior_alpha=None):
    if prior_alpha is None:
        prior_alpha = np.ones(6)
    observed_counts = np.bincount(observed_rolls, minlength=7)[1:]
    posterior_alpha = prior_alpha + observed_counts
    posterior_probs = posterior_alpha / posterior_alpha.sum()
    return posterior_probs, posterior_alpha

def calculate_expected_sum(observed_rolls, posterior_probs, total_rolls):
    # Expected value per remaining roll
    expected_value_per_roll = np.dot(np.arange(1, 7), posterior_probs)
    remaining_rolls = total_rolls - len(observed_rolls)
    # Expected total sum
    expected_sum = sum(observed_rolls) + remaining_rolls * expected_value_per_roll
    return expected_sum

def simulate_confidence_interval(observed_rolls, posterior_alpha, total_rolls, confidence_level=95, num_simulations=10000):
    remaining_rolls = total_rolls - len(observed_rolls)
    simulated_totals = []
    # Generate samples from the posterior Dirichlet distribution
    for _ in range(num_simulations):
        # Sample die weights from the posterior Dirichlet distribution
        sampled_probs = dirichlet.rvs(posterior_alpha, size=1)[0]
        # Simulate remaining rolls using sampled probabilities
        simulated_remaining_rolls = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs
        )
        # Calculate total sum
        simulated_total = sum(observed_rolls) + sum(simulated_remaining_rolls)
        simulated_totals.append(simulated_total)
    # Calculate confidence interval
    lower_bound = np.percentile(simulated_totals, (100 - confidence_level) / 2)
    upper_bound = np.percentile(simulated_totals, 100 - (100 - confidence_level) / 2)
    return lower_bound, upper_bound

def simulate_spread(observed_rolls_A, posterior_alphaA, observed_rolls_B, posterior_alphaB, total_rolls, confidence_levels=[95, 80, 60], num_simulations=10000):
    remaining_rolls = total_rolls - len(observed_rolls_A)  # Assuming same number of rolls for A and B

    simulated_spreads = []

    for _ in range(num_simulations):
        # Sample die probabilities for A and B
        sampled_probs_A = dirichlet.rvs(posterior_alphaA, size=1)[0]
        sampled_probs_B = dirichlet.rvs(posterior_alphaB, size=1)[0]
        # Simulate remaining rolls
        simulated_remaining_rolls_A = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs_A
        )
        simulated_remaining_rolls_B = np.random.choice(
            [1, 2, 3, 4, 5, 6],
            size=remaining_rolls,
            p=sampled_probs_B
        )
        # Compute total sums
        total_sum_A = sum(observed_rolls_A) + sum(simulated_remaining_rolls_A)
        total_sum_B = sum(observed_rolls_B) + sum(simulated_remaining_rolls_B)
        # Compute spread as the absolute difference
        spread = total_sum_A - total_sum_B
        simulated_spreads.append(spread)

    simulated_spreads = np.array(simulated_spreads)

    # Compute statistics
    expected_spread = np.mean(simulated_spreads)
    variance_spread = np.var(simulated_spreads)
    std_spread = np.std(simulated_spreads)

    # Since spread cannot be negative, lower bound is zero
    # Compute confidence intervals
    confidence_intervals = {}
    for cl in confidence_levels:
        lower_percentile = (100 - cl) / 2
        upper_percentile = 100 - lower_percentile
        lower_bound = np.percentile(simulated_spreads, lower_percentile)
        upper_bound = np.percentile(simulated_spreads, upper_percentile)
        confidence_intervals[cl] = (lower_bound, upper_bound)

    return expected_spread, variance_spread, std_spread, confidence_intervals

def main(teams, total_rolls, our_choiceA, our_choiceB, rounds_revealed, other_choices_probs, observed_rolls_A, observed_rolls_B):

    # Using a uniform prior (Dirichlet prior with alpha=1 for each face)
    posterior_probsA, posterior_alphaA = bayesian_update(observed_rolls_A)
    posterior_probsB, posterior_alphaB = bayesian_update(observed_rolls_B)

    print(f"Rolls A: {observed_rolls_A}")
    print(f"Rolls B: {observed_rolls_B}")
    # Step 5: Calculate expected sum based on updated die weights
    expected_sum_A = calculate_expected_sum(observed_rolls_A, posterior_probsA, total_rolls)
    expected_sum_B = calculate_expected_sum(observed_rolls_B, posterior_probsB, total_rolls)

    # Step 6: Simulate confidence intervals for SUMA and SUMB
    lower_bound95A, upper_bound95A = simulate_confidence_interval(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=95, num_simulations=10000
    )

    lower_bound80A, upper_bound80A = simulate_confidence_interval(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=80, num_simulations=10000
    )

    lower_bound60A, upper_bound60A = simulate_confidence_interval(
        observed_rolls_A, posterior_alphaA, total_rolls, confidence_level=60, num_simulations=10000
    )

    lower_bound95B, upper_bound95B = simulate_confidence_interval(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=95, num_simulations=10000
    )

    lower_bound80B, upper_bound80B = simulate_confidence_interval(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=80, num_simulations=10000
    )

    lower_bound60B, upper_bound60B = simulate_confidence_interval(
        observed_rolls_B, posterior_alphaB, total_rolls, confidence_level=60, num_simulations=10000
    )

    # Step 7: Simulate spread
    expected_spread, variance_spread, std_spread, confidence_intervals_spread = simulate_spread(
        observed_rolls_A, posterior_alphaA, observed_rolls_B, posterior_alphaB, total_rolls, confidence_levels=[95,80,60], num_simulations=10000)

    # Compute bid and ask prices
    z = 1.96  # Z-score for 95% confidence
    bid_price = max(expected_spread - z * std_spread, 0)
    ask_price = expected_spread + z * std_spread

    # Get confidence intervals for spread
    lower_bound95_spread, upper_bound95_spread = confidence_intervals_spread[95]
    lower_bound80_spread, upper_bound80_spread = confidence_intervals_spread[80]
    lower_bound60_spread, upper_bound60_spread = confidence_intervals_spread[60]

    return (
        expected_sum_A, expected_sum_B,
        lower_bound95A, upper_bound95A, lower_bound80A, upper_bound80A, lower_bound60A, upper_bound60A,
        lower_bound95B, upper_bound95B, lower_bound80B, upper_bound80B, lower_bound60B, upper_bound60B,
        expected_spread, variance_spread, std_spread,
        lower_bound95_spread, upper_bound95_spread,
        lower_bound80_spread, upper_bound80_spread,
        lower_bound60_spread, upper_bound60_spread,
        bid_price, ask_price
    )

teams = 60
(
    our_choiceA, our_choiceB, total_rolls, rounds_revealed,
    other_choices_probs, observed_rolls_A, observed_rolls_B
) = read_parametersA()

(
    expected_sum_A, expected_sum_B,
    lower_bound95A, upper_bound95A, lower_bound80A, upper_bound80A, lower_bound60A, upper_bound60A,
    lower_bound95B, upper_bound95B, lower_bound80B, upper_bound80B, lower_bound60B, upper_bound60B,
    expected_spread, variance_spread, std_spread,
    lower_bound95_spread, upper_bound95_spread,
    lower_bound80_spread, upper_bound80_spread,
    lower_bound60_spread, upper_bound60_spread,
    bid_price, ask_price
) = main(
    teams, total_rolls, our_choiceA, our_choiceB, rounds_revealed,
    other_choices_probs, observed_rolls_A, observed_rolls_B
)

sheet.update([[expected_sum_A]],'F8')

sheet.update([[lower_bound95A]], 'F13')
sheet.update([[upper_bound95A]],'H13')

sheet.update([[lower_bound80A]],'F19')
sheet.update([[upper_bound80A]], 'H19')

sheet.update([[lower_bound60A]],'F25')
sheet.update([[upper_bound60A]], 'H25')

sheet.update([[expected_sum_B]],'F30')

sheet.update([[lower_bound95B]], 'F35')
sheet.update([[upper_bound95B]],'H35')

sheet.update([[lower_bound80B]],'F41')
sheet.update([[upper_bound80B]], 'H41')

sheet.update([[lower_bound60B]],'F47')
sheet.update([[upper_bound60B]], 'H47')
#--------------------------------------------------
sheet.update([[expected_spread]], 'J30')
sheet.update([[variance_spread]], 'F53')
sheet.update([[std_spread]], 'F54')

sheet.update([[lower_bound95_spread]], 'J35')
sheet.update([[upper_bound95_spread]], 'L35')

sheet.update([[lower_bound80_spread]], 'J41')
sheet.update([[upper_bound80_spread]], 'L41')

sheet.update([[lower_bound60_spread]], 'J47')
sheet.update([[upper_bound60_spread]], 'L47')

sheet.update([[bid_price]], 'J50')
sheet.update([[ask_price]], 'K50')


Rolls A: [1, 1, 6, 6, 6, 3, 1, 1, 6, 1, 1, 6, 6]
Rolls B: [1, 1, 1, 6, 6, 3, 3, 1, 6, 6, 6, 6, 6]


{'spreadsheetId': '1J1naRqGMA9R7LmnuybaedTQTWltqvXeFpgsnHSFWyoU',
 'updatedRange': 'Sheet1!K50',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}